In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as st 
from tqdm import tqdm 
import os
os.chdir('D:\Shashi Katteri\Desktop\data sets to work on\Amazon Employee Access\Dataset_10')

In [2]:
df=pd.read_csv('Amazon_employee_access.csv')
df.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,target
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1


In [3]:
df1=df.astype(str)
df1['target']=df1['target'].astype(int)

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32769 entries, 0 to 32768
Data columns (total 10 columns):
RESOURCE            32769 non-null object
MGR_ID              32769 non-null object
ROLE_ROLLUP_1       32769 non-null object
ROLE_ROLLUP_2       32769 non-null object
ROLE_DEPTNAME       32769 non-null object
ROLE_TITLE          32769 non-null object
ROLE_FAMILY_DESC    32769 non-null object
ROLE_FAMILY         32769 non-null object
ROLE_CODE           32769 non-null object
target              32769 non-null int32
dtypes: int32(1), object(9)
memory usage: 2.4+ MB


In [5]:
df1.describe(include='object')

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
count,32769,32769,32769,32769,32769,32769,32769,32769,32769
unique,7518,4243,128,177,449,343,2358,67,343
top,4675,770,117961,118300,117878,118321,117906,290919,118322
freq,839,152,21407,4424,1135,4649,6896,10980,4649


In [6]:
feat=list(df1.columns)
feat.remove('target')
for i in feat:
    for j in feat:
        if (i!=j) and all(df1[i]==df1[j]):
            print (i,', ',j)
        elif (i!=j) and len(df1.groupby(i).size())==len(df1.groupby(j).size()):
            print (i,', ',j)
    feat.remove(i)
    
    
    
    


ROLE_CODE ,  ROLE_TITLE


In [7]:
len(df1.groupby(['ROLE_CODE','ROLE_TITLE']))==len(df1.groupby('ROLE_TITLE'))

True

This means Role code and role title columns probably contain the same information. So we can drop one of them. Here we will drop Role title

In [8]:
df1=df1.drop('ROLE_TITLE',axis=1)

In [9]:
from itertools import combinations

feat=list(df1.columns)
for i in tqdm(feat):
    for j in range(2,len(feat)):
        for k in (combinations(feat,j)):
            if len(df1.groupby(list(k)).size())==len(df1.groupby(i).size()):
                print(k,'-->',i)

 78%|███████▊  | 7/9 [01:15<00:21, 10.58s/it]

('ROLE_FAMILY', 'ROLE_CODE') --> ROLE_CODE


100%|██████████| 9/9 [01:35<00:00, 10.57s/it]


From here we can conclude that the data captured by role family is captured by role code. This means when we have role code column role family becomes redundant

In [10]:
df1=df1.drop('ROLE_FAMILY',axis=1)

In [11]:
df1.describe(include='object')

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE
count,32769,32769,32769,32769,32769,32769,32769
unique,7518,4243,128,177,449,2358,343
top,4675,770,117961,118300,117878,117906,118322
freq,839,152,21407,4424,1135,6896,4649


## Univariate Analysis

### RESOURCE

In [12]:
rec=df1.groupby('RESOURCE').apply(np.mean)
rec[rec['target']==0]


,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target
RESOURCE,,,,,,,,
1001,1.001000e+03,16244.0,1.179610e+05,1.180520e+05,1.203180e+05,1.179060e+05,1.187860e+05,0.0
100614,1.006140e+05,7408.0,1.179610e+05,1.179620e+05,1.183520e+05,2.409830e+05,1.183220e+05,0.0
100821,1.008210e+05,17514.0,1.179610e+05,1.183430e+05,1.217470e+05,1.179060e+05,1.183220e+05,0.0
102702,5.135105e+10,22997170.5,5.898056e+10,5.911256e+10,6.222456e+10,1.558111e+11,5.916606e+10,0.0
1032,1.032000e+03,4108.0,1.188870e+05,1.188880e+05,1.191360e+05,1.225640e+05,1.191390e+05,0.0
...,...,...,...,...,...,...,...,...
96520,9.652000e+04,278563.0,1.186020e+05,1.186030e+05,1.179410e+05,1.185680e+05,1.185700e+05,0.0
98394,9.839400e+04,4468.0,1.179610e+05,1.184130e+05,1.199680e+05,1.179060e+05,1.183220e+05,0.0
98433,9.843300e+04,3692.0,1.179610e+05,1.184130e+05,1.262290e+05,2.077090e+05,1.215960e+05,0.0


In [13]:
index2=rec[rec['target']!=1].index
(df1.groupby('RESOURCE').size()[index2]).count()

1134

In [14]:
index2=rec[rec['target']==0].index
(df1.groupby('RESOURCE').size()[index2]).count()

292

In [15]:
rec.count()

RESOURCE            7518
MGR_ID              7518
ROLE_ROLLUP_1       7518
ROLE_ROLLUP_2       7518
ROLE_DEPTNAME       7518
ROLE_FAMILY_DESC    7518
ROLE_CODE           7518
target              7518
dtype: int64

There are 6384 resources that are never denied access

292 resources which account for 327 records were always denied access. 24 of these resources have more than one occurance. These maybe highly confidential or highly secure or sensitive resources as access was never granted


### MGR_ID

In [16]:
mg=df1.groupby('MGR_ID').apply(np.mean)
mg[mg['target']==0]


,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target
MGR_ID,,,,,,,,
102234,75078.0,102234.0,121411.0,125018.0,138789.0,138790.0,117880.0,0.0
108732,79092.0,108732.0,119062.0,119091.0,118684.0,131688.0,122991.0,0.0
112106,43877.0,112106.0,118079.0,118080.0,117878.0,302452.0,130638.0,0.0
12265,32223.0,12265.0,118120.0,118121.0,117920.0,120978.0,119325.0,0.0
14163,33642.0,14163.0,5110.0,117954.0,117895.0,281735.0,118570.0,0.0
...,...,...,...,...,...,...,...,...
83733,75181.0,83733.0,118200.0,118201.0,117920.0,122957.0,121070.0,0.0
83905,73815.0,83905.0,118315.0,118316.0,118522.0,168284.0,119851.0,0.0
89065,20284.0,89065.0,118953.0,118954.0,117941.0,117886.0,117880.0,0.0


In [17]:
index3=mg[mg['target']==0].index
(df1.groupby('MGR_ID').size()[index3]).count()

68

In [18]:
(df1.groupby('MGR_ID').size()[index3]).sum()

121

In [19]:
(df1.groupby('MGR_ID').size()[index3]>1).sum()

22

In [20]:
df1.groupby('MGR_ID').size().count()

4243

Out of 4243 managers 68 manager ids have only 0 target. 22 out of these have more than one record which means more than one employee under these managers were denied access. There are 3 managers with more than 5 employees who have been denied access

### Rollup 1

In [21]:
roll1=df1.groupby('ROLE_ROLLUP_1').apply(np.mean)
roll1[roll1['target']==0]

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target
ROLE_ROLLUP_1,,,,,,,,
121411,75078.0,102234.0,121411.0,125018.0,138789.0,138790.0,117880.0,0.0


### Rollup2

In [22]:
roll2=df1.groupby('ROLE_ROLLUP_2').apply(np.mean)
roll2[roll2['target']==0]

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target
ROLE_ROLLUP_2,,,,,,,,
125018,75078.0,102234.0,121411.0,125018.0,138789.0,138790.0,117880.0,0.0
286791,80859.0,8030.0,117961.0,286791.0,286792.0,117906.0,117908.0,0.0


### ROLE_DEPTNAME

In [23]:
dept=df1.groupby('ROLE_DEPTNAME').apply(np.mean)
dept[dept['target']==0]

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target
ROLE_DEPTNAME,,,,,,,,
121169,1.572060e+09,2.553755e+07,5.898056e+10,5.915006e+10,6.058456e+10,1.390071e+11,5.926606e+10,0.0
138789,7.507800e+04,1.022340e+05,1.214110e+05,1.250180e+05,1.387890e+05,1.387900e+05,1.178800e+05,0.0
215920,7.984819e+18,1.895440e+15,2.974753e+22,2.974778e+22,5.398005e+22,7.639533e+22,3.073853e+22,0.0
286792,8.085900e+04,8.030000e+03,1.179610e+05,2.867910e+05,2.867920e+05,1.179060e+05,1.179080e+05,0.0


In [24]:
index4=dept[dept['target']==1].index
(df1.groupby('ROLE_DEPTNAME').size()[index4]).count()

182

In [25]:
(df1.groupby('ROLE_DEPTNAME').size()[index4]).sum()

3517

In 182 departments all employees have got access to resource they requested. It amounts to a total of 3517 employees

### ROLE_FAMILY_DESC

In [26]:
desc=df1.groupby('ROLE_FAMILY_DESC').apply(np.mean)
desc[desc['target']==0].count()

RESOURCE            46
MGR_ID              46
ROLE_ROLLUP_1       46
ROLE_ROLLUP_2       46
ROLE_DEPTNAME       46
ROLE_FAMILY_DESC    46
ROLE_CODE           46
target              46
dtype: int64

In [27]:
index5=desc[desc['target']==1].index
print((df1.groupby('ROLE_FAMILY_DESC').size()[index5]).sum())
print((df1.groupby('ROLE_FAMILY_DESC').size()[index5]).count())

10641
1974


In [28]:
index5=desc[desc['target']==0].index
(df1.groupby('ROLE_FAMILY_DESC').size()[index5]).sum()

65

A total of 1974 out of 2358 role descriptions have records with only 1 as target. This means these job family descriptions have been given access all the time for their corresponding resources

### ROLE_CODE

In [50]:
rcode=df1.groupby('ROLE_CODE').apply(np.mean)
rcode[rcode['target']==0].count()

RESOURCE            3
MGR_ID              3
ROLE_ROLLUP_1       3
ROLE_ROLLUP_2       3
ROLE_DEPTNAME       3
ROLE_FAMILY_DESC    3
ROLE_CODE           3
target              3
dtype: int64

In [51]:
index6=rcode[rcode['target']==0].index
(df1.groupby('ROLE_CODE').size()[index6]).sum()

4

## Bi-variate analysis

In [29]:
rollups=df1.groupby(['ROLE_ROLLUP_1','ROLE_ROLLUP_2']).apply(np.mean)

In [30]:
rollups[rollups.target!=1].count()

RESOURCE            108
MGR_ID              108
ROLE_ROLLUP_1       108
ROLE_ROLLUP_2       108
ROLE_DEPTNAME       108
ROLE_FAMILY_DESC    108
ROLE_CODE           108
target              108
dtype: int64

In [31]:
rollups.count()

RESOURCE            187
MGR_ID              187
ROLE_ROLLUP_1       187
ROLE_ROLLUP_2       187
ROLE_DEPTNAME       187
ROLE_FAMILY_DESC    187
ROLE_CODE           187
target              187
dtype: int64

In [32]:
rollups[rollups.target<0.8]

RESOURCE         MGR_ID  ROLE_ROLLUP_1  \
ROLE_ROLLUP_1 ROLE_ROLLUP_2                                                
117961        286791          8.085900e+04   8.030000e+03   1.179610e+05   
117978        119216          7.161295e+28   3.314281e+26   1.966302e+34   
118138        118139         1.793927e+104   2.398394e+96  5.625625e+123   
118212        119763         9.090335e+106  1.188330e+103  5.373278e+129   
              119836          4.332221e+27   2.872529e+28   1.970202e+34   
118256        118257                   inf            inf            inf   
119062        130600         2.657586e+137  5.984455e+120  4.252219e+165   
119920        119921         3.073357e+261  2.385599e+221            inf   
121411        125018          7.507800e+04   1.022340e+05   1.214110e+05   

                             ROLE_ROLLUP_2  ROLE_DEPTNAME  ROLE_FAMILY_DESC  \
ROLE_ROLLUP_1 ROLE_ROLLUP_2                                                   
117961        286791          2.867910e+05   2.867920e+05      1.179060e+05   
117978        119216          1.986935e+34   1.964735e+34      4.695585e+34   
118138        118139         5.625672e+123  5.613529e+123     5.614918e+123   
118212        119763         5.443778e+129  5.598778e+129     9.072373e+129   
              119836          1.997269e+34   1.997285e+34      2.340402e+34   
118256        118257                   inf            inf               inf   
119062        130600         4.664290e+165  4.988469e+165     4.210933e+165   
119920        119921                   inf            inf               inf   
121411        125018          1.250180e+05   1.387890e+05      1.387900e+05   

                                 ROLE_CODE    target  
ROLE_ROLLUP_1 ROLE_ROLLUP_2                           
117961        286791          1.179080e+05  0.000000  
117978        119216          1.976169e+34  0.500000  
118138        118139         5.613720e+123  0.761905  
118212        119763         5.660869e+129  0.727273  
              119836          1.974769e+34  0.500000  
118256        118257                   inf  0.719844  
119062        130600         4.211004e+165  0.500000  
119920        119921                   inf  0.775862  
121411        125018          1.178800e+05  0.000000

In [33]:
index1=rollups[rollups.target<0.8].index
df1.groupby(['ROLE_ROLLUP_1','ROLE_ROLLUP_2']).size()[index1]

ROLE_ROLLUP_1  ROLE_ROLLUP_2
117961         286791             1
117978         119216             6
118138         118139            21
118212         119763            22
               119836             6
118256         118257           257
119062         130600            28
119920         119921            58
121411         125018             1
dtype: int64

In [57]:
code_desc=df1.groupby(['ROLE_FAMILY_DESC','ROLE_CODE']).apply(np.mean)
code_desc['target'][code_desc['target']==0].count()

56

In [56]:
index=code_desc['target'][code_desc['target']==1].index
df1.groupby(['ROLE_FAMILY_DESC','ROLE_CODE']).size()[index].sum()

13180

In [58]:
code_desc.count()

RESOURCE            3013
MGR_ID              3013
ROLE_ROLLUP_1       3013
ROLE_ROLLUP_2       3013
ROLE_DEPTNAME       3013
ROLE_FAMILY_DESC    3013
ROLE_CODE           3013
target              3013
dtype: int64

In [59]:
code_dept=df1.groupby(['ROLE_DEPTNAME','ROLE_CODE']).apply(np.mean)
code_dept['target'][code_dept['target']==0].count()

40

In [60]:
code_dept.count()

RESOURCE            2946
MGR_ID              2946
ROLE_ROLLUP_1       2946
ROLE_ROLLUP_2       2946
ROLE_DEPTNAME       2946
ROLE_FAMILY_DESC    2946
ROLE_CODE           2946
target              2946
dtype: int64

In [38]:
role_rec=df1.groupby(['RESOURCE','ROLE_ROLLUP_1','ROLE_ROLLUP_1']).apply(np.mean)

In [39]:
index=role_rec[role_rec['target']==0].index

In [40]:
(df1.groupby(['RESOURCE','ROLE_ROLLUP_1','ROLE_ROLLUP_1']).size()>1).sum()

4592

In [41]:
df1.groupby(['RESOURCE','ROLE_ROLLUP_1','ROLE_ROLLUP_1']).size()[index].sum()

564

if we consider resource and both roll ups there are 504 combinations that are only present in target 0

A total of 564 0s can be predicted with this.

out of 11000 combinations only 4600 combinations have more than one occurance

In [35]:
newfeats=pd.DataFrame(df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2'],columns=['ROLLUP_1+2'])


In [36]:
RESOURCE_ACCESS_PROB=[]
newfeats['RESOURCE']=df1['RESOURCE']
for i in tqdm(newfeats.index):
    ind=newfeats.loc[i]['RESOURCE']
    RESOURCE_ACCESS_PROB.append(rec.loc[ind]['target'])
newfeats['RESOURCE_ACCESS_PROB']=RESOURCE_ACCESS_PROB

100%|██████████| 32769/32769 [00:08<00:00, 4088.56it/s]


In [37]:
newfeats.head()

,ROLLUP_1+2,RESOURCE,RESOURCE_ACCESS_PROB
0,117961118300,39353,1.000000
1,117961118343,17183,0.966667
2,118219118220,36724,1.000000
3,117961118343,36135,1.000000
4,117929117930,42680,1.000000


In [42]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score,RandomizedSearchCV,StratifiedKFold

In [43]:
df1.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target
0,39353,85475,117961,118300,123472,117906,117908,1
1,17183,1540,117961,118343,123125,118536,118539,1
2,36724,14457,118219,118220,117884,267952,117880,1
3,36135,5396,117961,118343,119993,240983,118322,1
4,42680,5905,117929,117930,119569,123932,119325,1


In [44]:
x=df1.drop('target',axis=1)
x=pd.get_dummies(x,drop_first=True)
y=df1['target'].map({1:0,0:1})

In [45]:
x.shape

(32769, 15209)

lgb=LGBMClassifier()
params={'num_leaves':st.randint(10,50),
    'max_depth':st.randint(5,50),
    'learning_rate':st.uniform(0,1)}
sf=StratifiedKFold(n_splits=5)
rs=RandomizedSearchCV(lgb,param_distributions=params,cv=sf,random_state=7,scoring='roc_auc')
m=rs.fit(x,y)
lbp=m.best_params_
print(m.best_score_)
l=LGBMClassifier(**lbp)
cv1=cross_val_score(l,x,y,scoring='roc_auc',cv=sf)
cv2=cross_val_score(l,x,y,scoring='accuracy',cv=sf)
cv3=cross_val_score(l,x,y,scoring='recall',cv=sf)
print('ROC_AUC: ',cv1.mean()),
print('Accuracy: ',cv2.mean())
print('Recall: ',cv3.mean())

from sklearn.naive_bayes import BernoulliNB
nb=BernoulliNB()
cv1=cross_val_score(nb,x,y,scoring='roc_auc',cv=sf)
cv2=cross_val_score(nb,x,y,scoring='accuracy',cv=sf)
cv3=cross_val_score(nb,x,y,scoring='recall',cv=sf)
print('ROC_AUC: ',cv1.mean()),
print('Accuracy: ',cv2.mean())
print('Recall: ',cv3.mean())